# Question 1

In [1]:
import pandas as pd
import time
import string
import time

Loading the password files

In [2]:
passwords1 = open(r"/Users/marcodicio/Desktop/Marco/DATA/passwords1.txt").read().splitlines()
passwords2 = open(r"/Users/marcodicio/Desktop/Marco/DATA/passwords2.txt").read().splitlines()
pass1 = passwords1
pass2 = passwords2


Defining the 2 hash functions to be used:

Method: For each existing string character i assign a value going from 0 to n (number of characters). Then i perform some transformations to make the hashes random. I finally mod with a prime number to minimise collisions and make hashes of similar length.


In [4]:
def myhash1(s):
   
    a = string.printable
    lst = []
    for i in a:
        lst.append(i)  
        
    encoder = []
    for j in range(len(lst)):
        encoder.append(int((((j+100)**2)*5059)+47))
            

    keys= lst
    values= encoder
    dictionary = dict(zip(keys, values))
   
    empty= []
    for e in s:
        empty.append(dictionary.get(e))
    str1 =  int(''.join(map(str, empty)))
    return str1%8114217931

def myhash2(s):
    
    a = string.printable
    lst = []
    for i in a:
        lst.append(i)  
        
    encoder = []
    for j in range(len(lst)):
        encoder.append(int(10 + ((j+100)**(3/2))*67))
            

    keys= lst
    values= encoder
    dictionary = dict(zip(keys, values))
   
    empty= []
    for e in s:
        empty.append(dictionary.get(e))
    str1 =  int(''.join(map(str, empty)))
    return str1%7076121391

# Hashing passwords. 

Running time note: I believe the hashing of the passwords should not be included in the bloom filter function since most passwords are stored in their hash equivalent and also hashing the whole database 1 time over actually takes a very long time.
The running time of my hash functions is between 0.0006 and 0.0008 per string. Total strings to hash are more than 100 million, with 2 hashes this takes about 6 to 8 hours. Hence the actual bloom filter will be used on the list of already hashed passwords.

Hashing existing passwords: passwords1

In [ ]:
hash1p1 = [myhash1(str(pass1[i])) for i in range(len(pass1))]
hash2p1 = [myhash2(str(pass1[i])) for i in range(len(pass1))]

Hashing all the possible new passwords: passwords2

In [ ]:
hash1p2 = [myhash1(str(pass2[i])) for i in range(len(pass2))]
hash2p2 = [myhash2(str(pass2[i])) for i in range(len(pass2))]

# Bloom Filter

Creating the bloom filter which is initially a containter list of all 0s. Length based on range of all possible hashes, determined by largest prime used in modding the hashes string.

In [ ]:
def BloomFilter(pass1,pass2):
    start = time.time()
    
    bf0 = [0]*8114217931
    hashlist = [hash1p1,hash2p1]
    listunion =  list(set().union(*hashlist))
    L = len(listunion)
    for i in range(L):
        bf0[listunion[i]] = 1
    newhashlist = [hash1p2,hash2p2]
    yesno= []
    for i in range(len(hash1p2)):
        if bf0[newhashlist[0][i]] == 1 and bf0[newhashlist[1][i] == 1]:
            yesno.append(1)
        else:
            yesno.append(0)
    end = time.time()
    
    
    #Printing question answers
    
    m = 8114217931#Length of bloom filter
    k = 2 #Number of hash functions used
    n = 100000000 #objects mapped in bloom filter(passwords1)
    p = (1-((1-1/m)**(k*n)))**k
  
    print('Number of hash function used: ', 2)
    print('Number of duplicates detected: ', sum(yesno))
    print('Probability of false positives: ', p)
    print('Execution time: ', end-start)



Bloom Filter steps:

1 - Initialising filter
2 - Joining the lists(set union) of the 2 hash functions applied to the existing passwords(passwords1)
3 - Filling the bloom filter with each index found by both hash functions
4 - Creating another list containing the prediction of our bloom filter
5 - Filling this list with 1s if both hashes of possible new passwords exist in bloom filter, 0 otherwise. 1 in "yesno" list means password from password2 database probably exists in password1 database already, 0 means password for sure does not exist.


Probability of false positives (FPR)
To calculate the probability of false positives i reference this source: https://tsapps.nist.gov/publication/get_pdf.cfm?pub_id=903775
I use Bloom's classical formulae for FPR. This underestimates the actual FPR of my bloom filter as proved in the paper but more accurate formulaes provided in the paper are computationally inefficient if not impossible with large datasets.